In [6]:
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
# https://towardsdatascience.com/how-to-create-and-customize-venn-diagrams-in-python-263555527305
from matplotlib_venn import venn2, venn2_circles
from matplotlib_venn import venn3, venn3_circles

import numpy as np

from extract.jobs import parse_mallob, get_baseline

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [52]:
plt.close("all")

In [7]:
static_2_df = pd.DataFrame(parse_mallob("benchmark/static_5_1_2_random"))
static_4_df = pd.DataFrame(parse_mallob("benchmark/static_4_1_4_new"))
static_8_df = pd.DataFrame(parse_mallob("benchmark/static_3_1_8_random"))

In [41]:
for df in [static_2_df, static_4_df, static_8_df]:
    df.loc[(df['result'] != "UNKNOWN") & (df['generate_duration'] < 0), 'generate_duration'] = 0

In [8]:
baseline_df = pd.DataFrame(get_baseline("benchmark/baseline.txt"))

In [10]:
two = static_2_df.set_index("identifier")["result"].rename("two")
four = static_4_df.set_index("identifier")["result"].rename("four")
eight = static_8_df.set_index("identifier")["result"].rename("eight")

result = pd.concat([two, four, eight], axis=1, join="outer").fillna("UNKNOWN")

result.groupby(['two', 'four', 'eight']).size().reset_index().rename(columns={0:'count'})

,two,four,eight,count
0,SAT,SAT,SAT,90
1,SAT,SAT,UNKNOWN,9
2,SAT,UNKNOWN,SAT,5
3,SAT,UNKNOWN,UNKNOWN,10
4,UNKNOWN,SAT,SAT,2
5,UNKNOWN,SAT,UNKNOWN,3
6,UNKNOWN,UNKNOWN,SAT,9
7,UNKNOWN,UNKNOWN,UNKNOWN,196
8,UNKNOWN,UNKNOWN,UNSAT,6
9,UNKNOWN,UNSAT,UNKNOWN,1


In [57]:
fig = plt.figure()

# Schnittmenge der gelösten (Löst es die gleichen Instanzen?)
venn3(subsets = (10 + 3, 3 + 1, 9 + 1, 9 + 6, 5 + 1, 2 + 3, 90 + 60), set_labels =('$J=2$', '$J=4$', '$J=8$'), set_colors=('#377eb8', '#ff7f00', '#e41a1c'))

fig.set_size_inches(w=3, h=2)

fig.savefig("scale/venn.pdf")
fig.savefig("scale/venn.pgf")

In [58]:
fig = plt.figure()
fig.subplots_adjust(bottom=0.2, left=0.175)

temp_series = static_2_df[static_2_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
ax = temp_series.plot(color="#377eb8", style="-.", figsize=(3,2))

temp_series = static_4_df[static_4_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#ff7f00", style=":", ax=ax)

temp_series = static_8_df[static_8_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#e41a1c", style="--", ax=ax)

temp_series = baseline_df[baseline_df["result"]!="UNKNOWN"].sort_values("duration").reset_index()["duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#f781bf", style="-", ax=ax)

ax.set_xlabel("run time in s")
ax.set_ylabel("\# solved problems in $\leq t$ s")

ax.set_xticks([0, 250, 500, 750, 1000])
ax.set_xlim(left=0, right=1000)
ax.set_yticks([0, 50, 100, 150, 200])
ax.set_ylim(bottom=0)

ax.grid(axis="y")

ax.legend(['$J=2$', '$J=4$', '$J=8$', 'Cadical'])

fig.savefig("scale/time.pdf")
fig.savefig("scale/time.pgf")

In [59]:
fig = plt.figure()
fig.subplots_adjust(bottom=0.2, left=0.175)
temp_series = static_2_df[static_2_df["generate_duration"]>0].sort_values("generate_duration").reset_index()["generate_duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
ax = temp_series.plot(color="#377eb8", style="-.", figsize=(3,2))

temp_series = static_4_df[static_4_df["generate_duration"]>0].sort_values("generate_duration").reset_index()["generate_duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#ff7f00", style=":", ax=ax)

temp_series = static_8_df[static_8_df["generate_duration"]>0].sort_values("generate_duration").reset_index()["generate_duration"]
temp_series = pd.Series(dict((v,k) for k,v in temp_series.iteritems()))
temp_series.plot(color="#e41a1c", style="--", ax=ax)

ax.set_xlabel("run time in s")
ax.set_ylabel("generated cubes in $\leq t$ s")

ax.set_xticks([0, 250, 500, 750, 1000])
ax.set_xlim(left=0, right=1000)
ax.set_yticks([0, 100, 200, 300, 400])
ax.set_ylim(bottom=0)

ax.grid(axis="y")

ax.legend(['$J=2$', '$J=4$', '$J=8$'])

fig.savefig("scale/generated.pdf")
fig.savefig("scale/generated.pgf")

In [73]:
baseline_df.astype({"identifier": int}).sort_values("identifier").reset_index()

,index,identifier,duration,result
0,28,1,1001.0,UNKNOWN
1,26,2,1001.0,UNKNOWN
2,11,3,177.0,UNSATISFIABLE
3,21,4,763.0,UNSATISFIABLE
4,27,5,1000.0,UNKNOWN
...,...,...,...,...
394,395,396,1000.0,UNKNOWN
395,396,397,1000.0,UNKNOWN
396,368,398,623.0,SATISFIABLE
397,398,399,1000.0,UNKNOWN


In [82]:
static_2_df.loc[(static_2_df["result"] != "UNKNOWN") & (temp_df["result"] == "UNKNOWN")]

,identifier,start_time,end_time,duration,result,start_generate_cubes,end_generate_cubes,cube_count,generate_duration,root_node,number_send_cubes,number_returned_cubes,size_of_used_cube,size_of_added_buffer
35,36,12645.891,12726.406,80.515,SAT,12646.0,12646.9,0,0.9,33,39,8,-1,-1
42,43,14234.775,14284.653,49.878,SAT,14234.9,14235.5,0,0.6,13,70,9,-1,-1
58,59,18812.807,18862.717,49.910,SAT,18812.8,18813.6,0,0.8,56,39,8,-1,-1
61,62,19862.734,20683.156,820.422,SAT,19862.8,19908.4,0,45.6,27,71,10,-1,-1
62,63,20626.107,21180.896,554.789,SAT,20626.3,20696.2,0,69.9,31,146,85,-1,-1
72,73,24193.811,24853.031,659.220,SAT,24193.9,24216.4,0,22.5,5,75,13,-1,-1
102,103,34344.121,34861.043,516.922,SAT,34344.2,34366.4,0,22.2,28,68,10,-1,-1
108,109,35731.062,35870.016,138.954,SAT,35731.1,35732.1,0,1.0,47,90,32,-1,-1
109,110,35870.016,36637.426,767.410,SAT,35870.1,35881.1,0,11.0,24,103,45,-1,-1
112,113,37028.852,37188.715,159.863,SAT,37029.0,37050.4,0,21.4,13,55,26,-1,-1


In [83]:
# make indexed in baseline also identifier based
temp_df = baseline_df.astype({"identifier": int}).sort_values("identifier").reset_index()

df.loc[(df['result'] != "UNKNOWN") & (df['generate_duration'] < 0), 'generate_duration'] = 0

# Gen SAT UNSAT New(that cadical did not solve)
print("J = 2")
print(len(static_2_df[static_2_df["generate_duration"]>0]))
print(len(static_2_df[static_2_df["result"]=="SAT"]))
print(len(static_2_df[static_2_df["result"]=="UNSAT"]))
print(len(static_2_df.loc[(static_2_df["result"] != "UNKNOWN") & (temp_df["result"] == "UNKNOWN")]))

print("\n")

print("J = 4")
print(len(static_4_df[static_4_df["generate_duration"]>0]))
print(len(static_4_df[static_4_df["result"]=="SAT"]))
print(len(static_4_df[static_4_df["result"]=="UNSAT"]))
print(len(static_4_df.loc[(static_4_df["result"] != "UNKNOWN") & (temp_df["result"] == "UNKNOWN")]))

print("\n")

print("J = 8")
print(len(static_8_df[static_8_df["generate_duration"]>0]))
print(len(static_8_df[static_8_df["result"]=="SAT"]))
print(len(static_8_df[static_8_df["result"]=="UNSAT"]))
print(len(static_8_df.loc[(static_8_df["result"] != "UNKNOWN") & (temp_df["result"] == "UNKNOWN")]))

J = 2
330
114
65
44


J = 4
362
104
65
34


J = 8
370
106
70
32
